# Universidad del Valle de Guatemala
## Security Data Science - 10
* Walter Danilo Saldaña Salguero - 19897
* Javier Alejandro Cotto Argueta - 19324

## Laboratorio 3: Detección de Malware

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np
import sklearn.cluster as cluster
from sklearn.cluster import KMeans
import itertools
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from os import listdir
import os

### Exploración de Datos
Mostramos algunos datos del data set

In [2]:
df = pd.read_csv('malware.csv')
df.sample(10)

.rsrc  .rsrcvAddress  .rsrcvSize  .rsrcrSize  KERNEL32.DLL  MSVCRT.dll  \
12    NaN            NaN         NaN         NaN           NaN           1   
36    1.0        16384.0       124.0       512.0           1.0           1   
4     1.0        16384.0       124.0       512.0           1.0           1   
25    1.0        16384.0       132.0       512.0           1.0           1   
15    1.0      2031616.0       912.0      1024.0           1.0           1   
6     NaN            NaN         NaN         NaN           NaN           1   
3     NaN            NaN         NaN         NaN           NaN           1   
35    NaN            NaN         NaN         NaN           1.0           1   
34    1.0        16384.0       128.0       512.0           1.0           1   
37    1.0        16384.0       128.0       512.0           1.0           1   

    SHELL32.dll  USER32.dll  WS2_32.dll  htons  ...  shutdown  htonl  ntohl  \
12          1.0         1.0           1    NaN  ...       NaN    NaN    NaN   
36          NaN         1.0           1    1.0  ...       NaN    NaN    NaN   
4           1.0         1.0           1    1.0  ...       NaN    NaN    NaN   
25          1.0         1.0           1    1.0  ...       NaN    NaN    NaN   
15          1.0         NaN           1    NaN  ...       NaN    NaN    NaN   
6           1.0         1.0           1    NaN  ...       NaN    NaN    NaN   
3           1.0         1.0           1    NaN  ...       NaN    NaN    NaN   
35          NaN         1.0           1    1.0  ...       1.0    1.0    1.0   
34          NaN         1.0           1    1.0  ...       NaN    NaN    NaN   
37          1.0         1.0           1    1.0  ...       NaN    NaN    NaN   

    select  WSACleanup  WSAGetLastError  PSAPI.DLL  listen  bind  accept  
12     NaN         NaN              NaN        NaN     NaN   NaN     NaN  
36     NaN         NaN              NaN        NaN     NaN   NaN     NaN  
4      NaN         NaN              NaN        NaN     NaN   NaN     NaN  
25     NaN         NaN              NaN        NaN     NaN   NaN     NaN  
15     NaN         NaN              NaN        NaN     NaN   NaN     NaN  
6      NaN         NaN              NaN        NaN     NaN   NaN     NaN  
3      NaN         NaN              NaN        NaN     NaN   NaN     NaN  
35     1.0         1.0              1.0        1.0     1.0   1.0     1.0  
34     NaN         NaN              NaN        NaN     NaN   NaN     NaN  
37     NaN         NaN              NaN        NaN     NaN   NaN     NaN  

[10 rows x 45 columns]

#### Cantidad de Filas y Columnas

In [3]:
df.shape

(40, 45)

#### Columnas, nulos y tipos de datos

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 45 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   .rsrc            25 non-null     float64
 1   .rsrcvAddress    25 non-null     float64
 2   .rsrcvSize       25 non-null     float64
 3   .rsrcrSize       25 non-null     float64
 4   KERNEL32.DLL     32 non-null     float64
 5   MSVCRT.dll       40 non-null     int64  
 6   SHELL32.dll      25 non-null     float64
 7   USER32.dll       37 non-null     float64
 8   WS2_32.dll       40 non-null     int64  
 9   htons            29 non-null     float64
 10  connect          29 non-null     float64
 11  socket           29 non-null     float64
 12  WSAStartup       40 non-null     int64  
 13  send             29 non-null     float64
 14  inet_addr        29 non-null     float64
 15  recv             29 non-null     float64
 16  closesocket      29 non-null     float64
 17  .data            1

#### Descripción estadistica de los datos

In [5]:
df.describe()

.rsrc  .rsrcvAddress   .rsrcvSize  .rsrcrSize  KERNEL32.DLL  \
count   25.0   2.500000e+01    25.000000        25.0          32.0   
mean     1.0   4.187750e+05   257.120000       614.4           1.0   
std      0.0   1.005161e+06   305.818072       256.0           0.0   
min      1.0   1.638400e+04   124.000000       512.0           1.0   
25%      1.0   1.638400e+04   124.000000       512.0           1.0   
50%      1.0   1.638400e+04   128.000000       512.0           1.0   
75%      1.0   1.638400e+04   132.000000       512.0           1.0   
max      1.0   4.030464e+06  1032.000000      1536.0           1.0   

       MSVCRT.dll  SHELL32.dll  USER32.dll  WS2_32.dll  htons  ...  shutdown  \
count        40.0         25.0        37.0        40.0   29.0  ...       8.0   
mean          1.0          1.0         1.0         1.0    1.0  ...       1.0   
std           0.0          0.0         0.0         0.0    0.0  ...       0.0   
min           1.0          1.0         1.0         1.0    1.0  ...       1.0   
25%           1.0          1.0         1.0         1.0    1.0  ...       1.0   
50%           1.0          1.0         1.0         1.0    1.0  ...       1.0   
75%           1.0          1.0         1.0         1.0    1.0  ...       1.0   
max           1.0          1.0         1.0         1.0    1.0  ...       1.0   

       htonl  ntohl  select  WSACleanup  WSAGetLastError  PSAPI.DLL  listen  \
count    8.0    8.0     8.0         8.0              8.0        5.0     2.0   
mean     1.0    1.0     1.0         1.0              1.0        1.0     1.0   
std      0.0    0.0     0.0         0.0              0.0        0.0     0.0   
min      1.0    1.0     1.0         1.0              1.0        1.0     1.0   
25%      1.0    1.0     1.0         1.0              1.0        1.0     1.0   
50%      1.0    1.0     1.0         1.0              1.0        1.0     1.0   
75%      1.0    1.0     1.0         1.0              1.0        1.0     1.0   
max      1.0    1.0     1.0         1.0              1.0        1.0     1.0   

       bind  accept  
count   2.0     2.0  
mean    1.0     1.0  
std     0.0     0.0  
min     1.0     1.0  
25%     1.0     1.0  
50%     1.0     1.0  
75%     1.0     1.0  
max     1.0     1.0  

[8 rows x 45 columns]

Con ayuda de Pandas Profiling nos ahorraremos varias lineas de código para realizar el analisis exploratorio del dataset

In [6]:
from pandas_profiling import ProfileReport
prof = ProfileReport(df)
prof

C:\Users\Javier Cotto\AppData\Local\Temp\ipykernel_17068\3036991050.py:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Implementación de K-Means

### Método del Codo

### Gráfica del error contra K(número de clusters)

### Cálculo del coeficiente de Silhouette